In [1]:
import re,codecs,random
import numpy as np
import pandas as pd
import os
import torch
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import argparse
import numpy as np
from collections import OrderedDict
from recordtype import recordtype
EntityInfo = recordtype('EntityInfo', 'type mstart mend sentNo')
PairInfo = recordtype('PairInfo', 'type direction cross closeA closeB')

In [3]:
def chunks(l, n):
    """
    Yield successive n-sized chunks from l.
    """
    for i in range(0, len(l), n):
        assert len(l[i:i + n]) == n
        yield l[i:i + n]

In [4]:
documents = {}
entities = {}
relations = {}
num_entity = {}
texts = []
labels = []
sents_train = []
num = 0
with open('train_filter.data', 'r') as infile:
    for line in infile:
        j=[]
        line = line.rstrip().split('\t')
        pairs = chunks(line[2:], 17)

        id_ = line[0]

        if id_ not in documents:
            documents[id_] = []

        for sent in line[1].split('|'):
            documents[id_] += [sent]
        for i in documents[id_]:
            j+= i.split(' ')
        if id_ not in entities:
            entities[id_] = OrderedDict()

        if id_ not in relations:
            relations[id_] = OrderedDict()

        for p in pairs:
            # pairs
            if (p[5], p[11]) not in relations[id_]:
                relations[id_][(p[5], p[11])] = PairInfo(p[0], p[1], p[2], p[3], p[4])
            else:
                print('duplicates!')
            ss = int(p[3].split('-')[0])
            se = int(p[3].split('-')[1])-1
            os = int(p[4].split('-')[0])
            oe = int(p[4].split('-')[1])-1
            sen = []
            for i_t, token in enumerate(j):
                if i_t == ss:
                    token = '[E1]'+ token
                if i_t == se:
                    token = token + '[/E1]'
                if i_t == os:
                    token = '[E2]' + token
                if i_t == oe:
                    token = token + '[/E2]'
                sen.append(token)
            tokens = (' ').join(sen)
            #texts.append(tokens)
            if p[0]!='1:CID:2':
                label=0
            else:
                label=1  
            # entities
            num = num + 1
            datacow = {'id':num,'token':tokens,'label':label}
            sents_train.append(datacow)

In [5]:
documents = {}
entities = {}
relations = {}
num_entity = {}
texts = []
labels = []
sents_test = []
num = 0
with open('test_filter.data', 'r') as infile:
    for line in infile:
        j=[]
        line = line.rstrip().split('\t')
        pairs = chunks(line[2:], 17)

        id_ = line[0]

        if id_ not in documents:
            documents[id_] = []

        for sent in line[1].split('|'):
            documents[id_] += [sent]
        for i in documents[id_]:
            j+= i.split(' ')
        if id_ not in entities:
            entities[id_] = OrderedDict()

        if id_ not in relations:
            relations[id_] = OrderedDict()

        for p in pairs:
            # pairs
            if (p[5], p[11]) not in relations[id_]:
                relations[id_][(p[5], p[11])] = PairInfo(p[0], p[1], p[2], p[3], p[4])
            else:
                print('duplicates!')
            ss = int(p[3].split('-')[0])
            se = int(p[3].split('-')[1])-1
            os = int(p[4].split('-')[0])
            oe = int(p[4].split('-')[1])-1
            sen = []
            for i_t, token in enumerate(j):
                if i_t == ss:
                    token = '[E1]'+ token
                if i_t == se:
                    token = token + '[/E1]'
                if i_t == os:
                    token = '[E2]' + token
                if i_t == oe:
                    token = token + '[/E2]'
                sen.append(token)
            tokens = (' ').join(sen)
            #texts.append(tokens)
            if p[0]!='1:CID:2':
                label=0
            else:
                label=1  
            # entities
            num = num + 1
            datacow = {'id':num,'token':tokens,'label':label}
            sents_test.append(datacow)

In [6]:
documents = {}
entities = {}
relations = {}
num_entity = {}
texts = []
labels = []
sents_dev = []
num = 0
with open('dev_filter.data', 'r') as infile:
    for line in infile:
        j=[]
        line = line.rstrip().split('\t')
        pairs = chunks(line[2:], 17)

        id_ = line[0]

        if id_ not in documents:
            documents[id_] = []

        for sent in line[1].split('|'):
            documents[id_] += [sent]
        for i in documents[id_]:
            j+= i.split(' ')
        if id_ not in entities:
            entities[id_] = OrderedDict()

        if id_ not in relations:
            relations[id_] = OrderedDict()

        for p in pairs:
            # pairs
            if (p[5], p[11]) not in relations[id_]:
                relations[id_][(p[5], p[11])] = PairInfo(p[0], p[1], p[2], p[3], p[4])
            else:
                print('duplicates!')
            ss = int(p[3].split('-')[0])
            se = int(p[3].split('-')[1])-1
            os = int(p[4].split('-')[0])
            oe = int(p[4].split('-')[1])-1
            sen = []
            for i_t, token in enumerate(j):
                if i_t == ss:
                    token = '[E1]'+ token
                if i_t == se:
                    token = token + '[/E1]'
                if i_t == os:
                    token = '[E2]' + token
                if i_t == oe:
                    token = token + '[/E2]'
                sen.append(token)
            tokens = (' ').join(sen)
            #texts.append(tokens)
            if p[0]!='1:CID:2':
                label=0
            else:
                label=1  
            # entities
            num = num + 1
            datacow = {'id':num,'token':tokens,'label':label}
            sents_dev.append(datacow)

In [7]:
from openprompt.data_utils import InputExample
dataset = {}
for split in ['train']:
    dataset[split] = []
    for data in sents_train:
        input_example = InputExample(text_a = data['token'], label=int(data['label']), guid=data['id'])
        dataset[split].append(input_example)
for split in ['test']:
    dataset[split] = []
    for data in sents_test:
        input_example = InputExample(text_a = data['token'], label=int(data['label']), guid=data['id'])
        dataset[split].append(input_example)
for split in ['validation']:
    dataset[split] = []
    for data in sents_dev:
        input_example = InputExample(text_a = data['token'], label=int(data['label']), guid=data['id'])
        dataset[split].append(input_example)

In [8]:
# You can load the plm related things provided by openprompt simply by calling:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")
#plm, tokenizer, model_config, WrapperClass = load_plm("gpt2", "gpt2")
# Constructing Template
# A template can be constructed from the yaml config, but it can also be constructed by directly passing arguments.
from openprompt.prompts import ManualTemplate

template_text1 = '{"placeholder":"text_a"} {"mask"}.'
template_text2 = '{"placeholder":"text_a"} is {"mask"}.'
template_text3 = '{"placeholder":"text_a"} The relation is {"mask"}.'
template_text4 = '{"placeholder":"text_a"} The relations between entities is {"mask"}.'

mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text1)

/root/miniconda3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 1024 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 1024 when padding/encoding.
- If you want to encode/pad to sequences longer than 1024 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [9]:
#from openprompt.plms import T5TokenizerWrapper
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=512, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

# You can see what a tokenized example looks like by 



In [10]:
model_inputs = {}
for split in ['train', 'validation', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

Token indices sequence length is longer than the specified maximum sequence length for this model (1094 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
from openprompt import PromptDataLoader
from openprompt.utils.metrics import classification_metrics
train_dataloader = PromptDataLoader(dataset=dataset['train'], template=mytemplate, tokenizer=tokenizer, 
    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, decoder_max_length=3, 
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=mytemplate, tokenizer=tokenizer, 
    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, decoder_max_length=3, 
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
test_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer, 
    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, decoder_max_length=3, 
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 5432it [00:13, 400.13it/s]
tokenizing: 5261it [00:12, 413.87it/s]
tokenizing: 5405it [00:13, 389.01it/s]


In [12]:
from openprompt.prompts import SoftVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
# myverbalizer = SoftVerbalizer(tokenizer, plm, num_classes=4,
#          label_words=["politics", "sports", "business", "technology"])
# or without label words
myverbalizer = SoftVerbalizer(tokenizer, plm, num_classes=2)

In [13]:
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and 
#print(myverbalizer.process_logits(logits)) # see what the verbalizer do

In [14]:
def evaluate(prompt_model, dataloader, desc):
    prompt_model.eval()
    allpreds = []
    alllabels = []
   
    for step, inputs in enumerate(dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        
        labels = inputs['label']
        alllabels.extend(labels.cpu().tolist())
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    score = classification_metrics(alllabels, allpreds, "micro-f1")
    return score

In [15]:
def evaluateprint(prompt_model, dataloader, desc):
    prompt_model.eval()
    allpreds = []
    alllabels = []
   
    for step, inputs in enumerate(dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        hidden = prompt_model.prompt_model(inputs)
        hidden = prompt_model.verbalizer.gather_outputs(hidden)
        outputs_at_mask = prompt_model.extract_at_mask(hidden, inputs)
        labels = inputs['label']
        alllabels.extend(labels.cpu().tolist())
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    score = classification_metrics(alllabels, allpreds, "micro-f1")
    return alllabels,allpreds,score

In [16]:
def evaluatetest(prompt_model, dataloader, desc):
    prompt_model.eval()
    allpreds = []
    alllabels = []
   
    for step, inputs in enumerate(dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        alllabels.extend(labels.cpu().tolist())
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    score = classification_metrics(alllabels, allpreds, "micro-f1")
    return alllabels,allpreds,score

In [17]:
# Although you can manually combine the plm, template, verbalizer together, we provide a pipeline 
# model which take the batched data from the PromptDataLoader and produce a class-wise logits

from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()

# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
# Although you can manually combine the plm, template, verbalizer together, we provide a pipeline 
# model which take the batched data from the PromptDataLoader and produce a class-wise logits

from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()

# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)


best_val_f1 = 0

leave_training = False

f1_traces = []

prompt_model.train()

for epoch in range(10):
    tot_loss = 0 
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step %1000 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)
    val_f1 = evaluate(prompt_model, validation_dataloader, desc="Valid")
    print("Epoch {}, val_acc: {}".format(epoch, val_f1), flush=True)
    if epoch > 0:
        
        if val_f1 >= best_val_f1:
            torch.save(prompt_model.state_dict(),f"tempalte1.ckpt")
            best_val_f1 = val_f1
            
        f1_traces.append(val_f1)
            #print("Glb_step {}, val_acc {}, average time {}".format(glb_step, val_acc, tot_train_time/actual_step ), flush=True)
    prompt_model.train()

Epoch 0, average loss: 0.713019847869873
Epoch 0, average loss: 0.49183981955616535
Epoch 0, val_acc: 0.8076411328644745
Epoch 1, average loss: 0.19491544365882874
Epoch 1, average loss: 0.45896347315085984
Epoch 1, val_acc: 0.8082113666603308
Epoch 2, average loss: 0.6699537932872772
Epoch 2, average loss: 0.3754803715571166
Epoch 2, val_acc: 0.778369131343851
Epoch 3, average loss: 0.20336908102035522
Epoch 3, average loss: 0.3093956710980785
Epoch 3, val_acc: 0.8378635240448584
Epoch 4, average loss: 0.13924746587872505
Epoch 4, average loss: 0.24219476047367927
Epoch 4, val_acc: 0.8490781220300323
Epoch 5, average loss: 0.21952487528324127
Epoch 5, average loss: 0.17710736055510815
Epoch 5, val_acc: 0.8093518342520434
Epoch 6, average loss: 0.07024658843874931
Epoch 6, average loss: 0.15264000511133835
Epoch 6, val_acc: 0.8327314198821516
Epoch 7, average loss: 0.019758321810513735
Epoch 7, average loss: 0.11675445420749786
Epoch 7, val_acc: 0.8296901729709181
Epoch 8, average loss

In [19]:
prompt_model.load_state_dict(torch.load(f"tempalte1.ckpt"))
prompt_model = prompt_model.cuda()
a,b,test_f1 = evaluatetest(prompt_model, test_dataloader, desc="Test")

In [20]:
from sklearn.metrics import matthews_corrcoef, f1_score, precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_pred=a, y_true=b, average="macro")

In [21]:
p

0.6814722441904157

In [22]:
r

0.7730290399744464

In [23]:
f

0.709896957942898